In [3]:
import requests
import json
endpoint = "https://1c4p8zfk1c.execute-api.eu-central-1.amazonaws.com/prod/askme"
question = "Hello"
answer = requests.post(url=endpoint, data=json.dumps({"body": question}))
answer.content

b'Unfortunately, I am unable to provide an accurate answer to the question "Hello" based on the context information provided. The context information includes details about Ulta Beauty\'s financial statements, such as deferred revenue, breakage on gift cards, and changes in deferred revenue. However, there is no information directly relevant to responding to simply "Hello". Without any additional details or context around what specifically is being asked in relation to the "Hello", I do not have enough information to provide a substantive response. I apologize that I am unable to answer this question accurately given the information provided. Please let me know if you need any clarification or have additional details to add to the question that could help me formulate an appropriate response.'

In [60]:
import boto3
import json

import os
import time
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth



In [61]:
session = boto3.Session(region_name="eu-central-1")
bedrock = session.client(
    "bedrock-runtime",
    "eu-central-1",
)


# instantiating the OpenSearch client, and passing in the CLI profile
opensearch = session.client("opensearchserverless")

In [73]:
host = "swmafrhpqvoso9v317ve.eu-central-1.aoss.amazonaws.com"
region = "eu-central-1"
service = "aoss"
credentials = session.get_credentials()

auth = AWSV4SignerAuth(credentials, region, service)

In [74]:
client = OpenSearch(
    hosts=[{"host": host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    pool_maxsize=20,
)

In [64]:
index_name = "vector"
search_query = {
    "size": 1,
    "query": {
        "match_all": {}
    }
}
response = client.search(index=index_name, body=search_query)
test_embedding = response["hits"]["hits"][1]['_source']['null']
print(test_embedding)


IndexError: list index out of range

In [65]:





def get_embedding(body):
    """
    This function is used to generate the embeddings for each question the user submits.
    :param body: This is the question that is passed in to generate an embedding
    :return: A vector containing the embeddings of the passed in content
    """
    # defining the embeddings model
    modelId = "amazon.titan-embed-text-v1"
    accept = "application/json"
    contentType = "application/json"
    # invoking the embedding model
    response = bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    # reading in the specific embedding
    response_body = json.loads(response.get("body").read())
    embedding = response_body.get("embedding")
    return embedding


def answer_query(user_input):
    """
    This function takes the user question, creates an embedding of that question,
    and performs a KNN search on your Amazon OpenSearch Index. Using the most similar results it feeds that into the Prompt
    and LLM as context to generate an answer.
    :param user_input: This is the natural language question that is passed in through the app.py file.
    :return: The answer to your question from the LLM based on the context that was provided by the KNN search of OpenSearch.
    """
    # Setting primary variables, of the user input
    userQuery = user_input
    # formatting the user input
    userQueryBody = json.dumps({"inputText": userQuery})
    # creating an embedding of the user input to perform a KNN search with
    userVectors = get_embedding(userQueryBody)
    print(userVectors)
    # the query parameters for the KNN search performed by Amazon OpenSearch with the generated User Vector passed in.
    # TODO: If you wanted to add pre-filtering on the query you could by editing this query!
    query = {
        "size": 3,
        "query": {"knn": {"vectors": {"vector": userVectors, "k": 3}}},
        "_source": True,
        "fields":["text"]
        
    }
    # performing the search on OpenSearch passing in the query parameters constructed above
    response = client.search(body=query, index=os.getenv("vector_index_name", "vector"))
    print(f"OpenSearch Search Response: {response}")

    # Format Json responses into text
    similaritysearchResponse = ""
    # iterating through all the findings of Amazon openSearch and adding them to a single string to pass in as context
    for i in response["hits"]["hits"]:
        outputtext = i["fields"]["text"]
        similaritysearchResponse = (
            similaritysearchResponse + "Info = " + str(outputtext)
        )

        similaritysearchResponse = similaritysearchResponse
    print(f"The context is: {similaritysearchResponse}")
    # Configuring the Prompt for the LLM
    # TODO: EDIT THIS PROMPT TO OPTIMIZE FOR YOUR USE CASE
    prompt_data = f"""\n\nHuman: You are an AI assistant that will help people answer questions they have. Answer the provided question to the best of your ability using the information provided in the Context. 
    Summarize the answer and provide sources to where the relevant information can be found. 
    Include this at the end of the response.
    Provide information based on the context provided.
    Format the output in human readable format - use paragraphs and bullet lists when applicable
    Answer in detail with no preamble
    If you are unable to answer accurately, please say so.
    Please mention the sources of where the answers came from by referring to page numbers, specific books and chapters!

    Question: {userQuery}

    Here is the text you should use as context: {similaritysearchResponse}

    \n\nAssistant:

    """
    # Configuring the model parameters, preparing for inference
    # TODO: TUNE THESE PARAMETERS TO OPTIMIZE FOR YOUR USE CASE
    prompt = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature": 0.5,
        "messages": [
            {"role": "user", "content": [{"type": "text", "text": prompt_data}]}
        ],
    }
    # formatting the prompt as a json string
    json_prompt = json.dumps(prompt)
    # invoking Claude3, passing in our prompt
    response = bedrock.invoke_model(
        body=json_prompt,
        modelId="anthropic.claude-v2:1",
        accept="application/json",
        contentType="application/json",
    )
    # getting the response from Claude3 and parsing it to return to the end user
    response_body = json.loads(response.get("body").read())
    # the final string returned to the end user
    answer = response_body["content"][0]["text"]
    # returning the final string to the end user
    
    return answer

user_input = "Tell me about the company"
answer= answer_query(user_input=user_input)
print(answer)

[-0.640625, -0.42578125, 0.32226562, 0.008972168, 0.48242188, 0.4140625, -0.22753906, -0.00030708313, -0.82421875, 0.6640625, -0.32421875, -0.5, 0.23925781, 0.011352539, 0.6640625, -0.33007812, 0.24121094, 0.31835938, 0.063964844, -0.032226562, 0.13085938, -0.48046875, 0.20214844, -0.375, 0.37109375, -0.064941406, -0.08935547, -0.4921875, -0.26953125, 0.16210938, -0.125, 0.34179688, -0.19824219, -0.4375, 0.34765625, -0.28125, 0.21289062, 0.390625, 0.09765625, 0.28320312, 0.72265625, 1.03125, 0.33007812, 0.080566406, 0.3203125, -0.484375, 0.625, -0.27148438, 0.25, 0.37109375, 0.49023438, -0.075683594, 0.8203125, 0.091308594, 0.3125, -0.6796875, -0.28515625, -0.72265625, -0.53515625, -0.40625, 0.90234375, 0.6015625, 0.56640625, -0.021606445, 0.15527344, -0.46289062, -0.32421875, -0.38671875, 0.45703125, -0.49609375, -0.13183594, 0.609375, 0.34960938, 0.4453125, 0.7578125, 0.064453125, 0.25976562, -0.088378906, 0.484375, 0.17382812, 0.114746094, -0.36328125, -0.1875, 0.06347656, -0.472656

In [75]:
client.indices.delete("vector")

{'acknowledged': True}

In [76]:
index_name = "vector"
index_body = {
    "settings": {"index": {"knn": True}},
    "mappings": {
        "properties": {
            "vectors": {
                "type": "knn_vector",
                "dimension": 1536,
                "method": {
                    "engine": "nmslib",
                    "name": "hnsw",
                },
            }
        }
    },
}
client.indices.create(index=index_name,body=index_body)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'vector'}

In [71]:
import requests
endpoint = "https://r9cvhvgvhc.execute-api.eu-central-1.amazonaws.com/prod/"
answer = requests.post(
                url=endpoint,
                json={
                    "query": "hi",
                    "user_context": "test",
                },
            )

In [72]:
answer.content

b'{"message": "Internal server error"}'